本notebook用于介绍`wandb`的大致用法, 同时结合`hydra`库打造一个简单机器学习训练demo

# Quick Start

从`conf`文件夹下读取配置

In [14]:
import wandb
import hydra
import os
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

# 这种方式不能解析命令行参数, 并且也不能使用 multi-run功能
# 使用 @hydra.main 来启用hydra
def load_config(config_path:str, config_name:str):
    with initialize(config_path=config_path, version_base="1.1"):
        return compose(config_name=config_name)

config = load_config(config_path="conf", config_name="default")
print(config)
print(OmegaConf.to_yaml(config))

{'project_name': 'wandb_demo', 'model': {'name': 'resnet18', 'depth': 18, 'num_classes': 10}, 'dataset': {'epochs': 10, 'batch_size': 32}, 'optimizer': {'lr': 0.005, 'beta1': 0.9, 'beta2': 0.999}}
project_name: wandb_demo
model:
  name: resnet18
  depth: 18
  num_classes: 10
dataset:
  epochs: 10
  batch_size: 32
optimizer:
  lr: 0.005
  beta1: 0.9
  beta2: 0.999



使用[wandb.init()](https://docs.wandb.ai/ref/python/sdk/functions/init/)来初始化wandb库, 初次使用需要输入[api_key](https://wandb.ai/authorize)   
- 使用`wandb.finish()`来停止

In [15]:
run = wandb.init(
    project=config.project_name, 
    entity="rem1-opera",     # username or team name
    name="demo",    # short display name
    config=OmegaConf.to_container(config, resolve=True),
    tags=["baseline"],
    group="test", 
)

accuracy,▁▃▆▇▇▇▇█▇
loss,█▅▅▁▂▁▂▁▁
accuracy,0.85796
loss,0.13384


启用后会看到如下界面

<img src="./img/wandb_init.png" width="80%" style="display:block; margin:auto;" />

In [16]:
import random

offset = random.random() / 5
for epoch in range(1, config.dataset.epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    wandb.log({"accuracy": acc, "loss": loss})  # 记录每个 epoch 的 acc 和 loss

run.finish()

epoch=1, accuracy=-0.1730977687092166, loss=1.3949701669801216
epoch=2, accuracy=0.2473615518816463, loss=0.42244520478292846
epoch=3, accuracy=0.6011621065565004, loss=0.2375206881122959
epoch=4, accuracy=0.6528086085072703, loss=0.1111705243821583
epoch=5, accuracy=0.7547778517406427, loss=0.08820603037325334
epoch=6, accuracy=0.8884041226510352, loss=0.14574586378051904
epoch=7, accuracy=0.9563369420783331, loss=0.17756758081716023
epoch=8, accuracy=0.9441312663440764, loss=0.10719346494383178
epoch=9, accuracy=0.8764487497298249, loss=0.08389733183686776


accuracy,▁▄▆▆▇████
loss,█▃▂▁▁▁▂▁▁
accuracy,0.87645
loss,0.0839


反映在面板上会有以下的记录曲线, 每次运行都有生成新的曲线:   

<img src="./img/wandb_run.png" width="80%" style="display:block; margin:auto;" />

可以在`workspace`面板下对比不同实验的指标情况:  

<img src="./img/differ.png" width="80%" style="display:block; margin:auto;" />